# Testing different features with RandomForest

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import bz2
import os
import sys
import nltk
import pickle
import pandas as pd
from IPython.display import display
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit

# Caching stopwords
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

sys.path.append(os.path.abspath('../src'))
from fact_classification import *

Load the dataset.

In [3]:
df, df_crowdsourced, df_ground_truth = data_loading(local=True)
df = df.dropna().reset_index(drop=True)
df.head()

,Sentence_id,Text,Speaker,Speaker_title,Speaker_party,File_id,Length,Line_number,Sentiment,Verdict
0,16,I think we've seen a deterioration of values.,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,8,16,0.000000,-1
1,17,I think for a while as a nation we condoned th...,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,16,17,-0.456018,-1
2,18,"For a while, as I recall, it even seems to me ...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,29,18,-0.805547,-1
3,19,"So we've seen a deterioration in values, and o...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,35,19,0.698942,-1
4,20,"We got away, we got into this feeling that val...",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,15,20,0.000000,-1


Load the baseline model scoring results. We will use this to compare against our new models.

In [4]:
# Load scoring results dataframe from the baseline model
df_score_train, df_score_test = score_loading()
df_score_train = df_score_train.reset_index(drop=True).drop(1)
df_score_test = df_score_test.reset_index(drop=True).drop(1)
print('Baseline model training scores:')
display(df_score_train)
print('Baseline model testing scores:')
display(df_score_test)

Baseline model training scores:


,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Baseline model testing scores:


,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,0.67,0.6,0.81,0.7,0.99,0.06,0.23,0.68,0.8,0.11,0.35,0.6


Load the features matrix that we generated in the `feature_generation.ipynb` notebook. This is a large sparse matrix. We convert it to Pandas.SparseDtype to avoid running out of memory when fitting our models.

In [5]:
with bz2.open('../results/df_features.bz2') as f:
    df_features = pickle.load(f)

df_features.head()

,Sentiment,Length,T_00,T_000,T_10,T_100,T_1000,T_101,T_104,T_105,...,E_money,E_norp,E_ordinal,E_org,E_percent,E_person,E_product,E_quantity,E_time,E_work_of_art
0,0.000000,-0.776708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.456018,-0.147669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.805547,0.874519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.698942,1.346298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,-0.226299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Splitting data and generating indexes

We split the dataset according to the instructions in the assignment, where data up until and including year 2008 will be used for training, and data after 2008 will be used for testing. Here we also generate indexes for the various feature sets.

In [6]:
df_train, df_test, idx_train = test_train_split(df)

# Column index for the numeric columns Sentiment and Length
col_idx_n = (df_features.columns == 'Sentiment') | (df_features.columns == 'Length')

# Column index for TF-IDF features on the raw Text column
col_idx_t = df_features.columns.str.startswith('T_')

# Column index for TF-IDF features on the stemmed text
col_idx_w = df_features.columns.str.startswith('W_')

# Column index for POS features
col_idx_p = df_features.columns.str.startswith('P_')

# Column index for NER labels
col_idx_e = df_features.columns.str.startswith('E_')

### Setup model for GridSearchCV

We use GridSearchCV from scikit-learn to get the best combination of parameters for the model. Since we have an unbalanced dataset we set `class_weight='balanced_subsample'` for the RandomForestClassifier. This will ensure that we have a balanced distribution in the subtrees.

For the grid search we will test with 100, 500, and 1000 trees, and max depths of 5, 10, and 20. We limit the depth do avoid overfitting, and we limit the number of trees to reduce training time. Further, we use weighted f1-score for scoring the grid search, and we define a test-fold for cross validation that uses the data after 2008 as test set.

In [7]:
rfc = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,
    class_weight='balanced_subsample'
)

param_grid = {
    'n_estimators': [500, 1000, 1500],
    'max_depth': [10, 20, 30]
}

# Define our own split for cross-validation. 
# A value of -1 excludes that index from testing, 
# and value 0 defines that index to test set 0. 
test_fold = np.full(len(df), -1)
test_fold[~idx_train] = 0
test_fold = PredefinedSplit(test_fold)

clf = GridSearchCV(
    estimator=rfc,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=test_fold,
    # n_jobs=-1,
    return_train_score=True,
    verbose=3
)


## RandomForest with only Length and Sentiment features

Using Sentiment and Length because these have the highest correlation in the original dataset.

In [8]:
# Index for the two numeric columns Length and Sentiment
idx_n = col_idx_n

# Fit model
clf_n = clf.fit(df_features.loc[:, idx_n], df['Verdict'])


Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV 1/1] END max_depth=10, n_estimators=500;, score=(train=0.571, test=0.470) total time=   2.8s
[CV 1/1] END max_depth=10, n_estimators=1000;, score=(train=0.572, test=0.470) total time=   2.9s
[CV 1/1] END max_depth=10, n_estimators=1500;, score=(train=0.571, test=0.470) total time=   4.0s
[CV 1/1] END max_depth=20, n_estimators=500;, score=(train=0.800, test=0.452) total time=   2.7s
[CV 1/1] END max_depth=20, n_estimators=1000;, score=(train=0.800, test=0.451) total time=   5.3s
[CV 1/1] END max_depth=20, n_estimators=1500;, score=(train=0.800, test=0.449) total time=   7.8s
[CV 1/1] END max_depth=30, n_estimators=500;, score=(train=0.800, test=0.450) total time=   2.8s
[CV 1/1] END max_depth=30, n_estimators=1000;, score=(train=0.800, test=0.449) total time=   5.6s
[CV 1/1] END max_depth=30, n_estimators=1500;, score=(train=0.800, test=0.448) total time=   8.4s


In [9]:
# Print the best parameters found
clf_n.best_params_

{'max_depth': 10, 'n_estimators': 1500}

In [10]:
df_pred_train_n = clf_n.predict(df_features.loc[idx_train, idx_n])
df_pred_test_n = clf_n.predict(df_features.loc[~idx_train, idx_n])

In [11]:
df_score_train = pd.concat([
    df_score_train,
    score_it(df_train['Verdict'], df_pred_train_n, algorithm='RFC', features='N')]
).reset_index(drop=True)

df_score_test = pd.concat([
    df_score_test,
    score_it(df_test['Verdict'], df_pred_test_n, algorithm='RFC', features='N')]
).reset_index(drop=True)

In [12]:
df_score_train

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
1,RFC,N,0.851,0.223,0.405,0.684,0.488,0.527,0.658,0.532,0.621,0.314,0.501,0.562


In [13]:
df_score_test

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,0.670,0.600,0.810,0.700,0.990,0.060,0.230,0.68,0.800,0.110,0.350,0.60
1,RFC,N,0.782,0.213,0.479,0.636,0.505,0.554,0.539,0.52,0.614,0.308,0.508,0.55


Using only Sentiment and Length gets very low accuracy, even lower than our baseline model.

Save the trained model to disk.

In [14]:
with bz2.open('../models/rfc_n.bz2', 'wb') as f:
    pickle.dump(clf_n, f)

### Using Length, Sentiment and stemmed text:

In [15]:
# Index for numeric (Length and Sentiment) and TF-IDF from stemmed text
idx_nw = col_idx_n | col_idx_w

# Fit model
clf_nw = clf.fit(df_features.loc[:, idx_nw], df['Verdict'])


Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV 1/1] END max_depth=10, n_estimators=500;, score=(train=0.726, test=0.641) total time=   4.6s
[CV 1/1] END max_depth=10, n_estimators=1000;, score=(train=0.723, test=0.638) total time=   5.5s
[CV 1/1] END max_depth=10, n_estimators=1500;, score=(train=0.719, test=0.635) total time=   6.1s
[CV 1/1] END max_depth=20, n_estimators=500;, score=(train=0.777, test=0.648) total time=   4.8s
[CV 1/1] END max_depth=20, n_estimators=1000;, score=(train=0.777, test=0.650) total time=   5.8s
[CV 1/1] END max_depth=20, n_estimators=1500;, score=(train=0.777, test=0.648) total time=   6.6s
[CV 1/1] END max_depth=30, n_estimators=500;, score=(train=0.832, test=0.655) total time=   5.3s
[CV 1/1] END max_depth=30, n_estimators=1000;, score=(train=0.833, test=0.657) total time=   6.5s
[CV 1/1] END max_depth=30, n_estimators=1500;, score=(train=0.832, test=0.654) total time=   7.4s


In [16]:
# Print the best parameters found
clf_nw.best_params_

{'max_depth': 30, 'n_estimators': 1000}

In [17]:
df_pred_train_nw = clf_nw.predict(df_features.loc[idx_train, idx_nw])
df_pred_test_nw = clf_nw.predict(df_features.loc[~idx_train, idx_nw])

In [18]:
df_score_train = pd.concat([
    df_score_train,
    score_it(df_train['Verdict'], df_pred_train_nw, algorithm='RFC', features='N_W')]
).reset_index(drop=True)

df_score_test = pd.concat([
    df_score_test,
    score_it(df_test['Verdict'], df_pred_test_nw, algorithm='RFC', features='N_W')]
).reset_index(drop=True)

In [19]:
df_score_train

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
1,RFC,N,0.851,0.223,0.405,0.684,0.488,0.527,0.658,0.532,0.621,0.314,0.501,0.562
2,RFC,N_W,0.936,0.490,0.739,0.845,0.808,0.861,0.785,0.808,0.868,0.625,0.761,0.818


In [20]:
df_score_test

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,0.670,0.600,0.810,0.700,0.990,0.060,0.230,0.680,0.800,0.110,0.350,0.600
1,RFC,N,0.782,0.213,0.479,0.636,0.505,0.554,0.539,0.520,0.614,0.308,0.508,0.550
2,RFC,N_W,0.877,0.439,0.809,0.808,0.816,0.825,0.630,0.767,0.845,0.573,0.708,0.777


We see that we have a small improvement in the weighted f1-score.

In [21]:
# Save trained classifier to disk
with bz2.open('../models/rfc_nw.bz2', 'wb') as f:
    pickle.dump(clf_nw, f)

### Using Length, Sentiment, stemmed text, and POS tags:

In [22]:
# Index for numeric (Length and Sentiment) and TF-IDF from stemmed text
idx_nwp = col_idx_n | col_idx_w | col_idx_p

# Fit model
clf_nwp = clf.fit(df_features.loc[:, idx_nwp], df['Verdict'])

Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV 1/1] END max_depth=10, n_estimators=500;, score=(train=0.758, test=0.679) total time=   4.4s
[CV 1/1] END max_depth=10, n_estimators=1000;, score=(train=0.764, test=0.682) total time=   5.3s
[CV 1/1] END max_depth=10, n_estimators=1500;, score=(train=0.763, test=0.685) total time=   5.8s
[CV 1/1] END max_depth=20, n_estimators=500;, score=(train=0.837, test=0.682) total time=   5.0s
[CV 1/1] END max_depth=20, n_estimators=1000;, score=(train=0.838, test=0.685) total time=   6.3s
[CV 1/1] END max_depth=20, n_estimators=1500;, score=(train=0.838, test=0.683) total time=   7.0s
[CV 1/1] END max_depth=30, n_estimators=500;, score=(train=0.909, test=0.687) total time=   6.0s
[CV 1/1] END max_depth=30, n_estimators=1000;, score=(train=0.910, test=0.688) total time=   7.4s
[CV 1/1] END max_depth=30, n_estimators=1500;, score=(train=0.909, test=0.686) total time=   9.1s


In [23]:
clf_nwp.best_params_

{'max_depth': 30, 'n_estimators': 1000}

In [24]:
df_pred_train_nwp = clf_nwp.predict(df_features.loc[idx_train, idx_nwp])
df_pred_test_nwp = clf_nwp.predict(df_features.loc[~idx_train, idx_nwp])

In [25]:
df_score_train = pd.concat([
    df_score_train,
    score_it(df_train['Verdict'], df_pred_train_nwp, algorithm='RFC', features='N_W_P')]
).reset_index(drop=True)

df_score_test = pd.concat([
    df_score_test,
    score_it(df_test['Verdict'], df_pred_test_nwp, algorithm='RFC', features='N_W_P')]
).reset_index(drop=True)

In [26]:
df_score_train

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
1,RFC,N,0.851,0.223,0.405,0.684,0.488,0.527,0.658,0.532,0.621,0.314,0.501,0.562
2,RFC,N_W,0.936,0.490,0.739,0.845,0.808,0.861,0.785,0.808,0.868,0.625,0.761,0.818
3,RFC,N_W_P,0.953,0.709,0.847,0.903,0.911,0.909,0.849,0.896,0.932,0.797,0.848,0.898


In [27]:
df_score_test

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,0.670,0.600,0.810,0.700,0.990,0.060,0.230,0.680,0.800,0.110,0.350,0.600
1,RFC,N,0.782,0.213,0.479,0.636,0.505,0.554,0.539,0.520,0.614,0.308,0.508,0.550
2,RFC,N_W,0.877,0.439,0.809,0.808,0.816,0.825,0.630,0.767,0.845,0.573,0.708,0.777
3,RFC,N_W_P,0.918,0.643,0.905,0.883,0.920,0.873,0.759,0.872,0.919,0.741,0.826,0.874


Including the POS-tagging features we get another small improvement on the weighted f1-score.

In [28]:
# Save trained classifier to disk
with bz2.open('../models/rfc_nwp.bz2', 'wb') as f:
    pickle.dump(clf_nwp, f)

### Using Length, Sentiment, stemmed text, POS tags, and NER labels:

In [29]:
# Index for numeric (Length and Sentiment) and TF-IDF from stemmed text
idx_nwpe = col_idx_n | col_idx_w | col_idx_p | col_idx_e

# Fit model
clf_nwpe = clf.fit(df_features.loc[:, idx_nwpe], df['Verdict'])

Fitting 1 folds for each of 9 candidates, totalling 9 fits
[CV 1/1] END max_depth=10, n_estimators=500;, score=(train=0.767, test=0.689) total time=   4.6s
[CV 1/1] END max_depth=10, n_estimators=1000;, score=(train=0.767, test=0.690) total time=   5.3s
[CV 1/1] END max_depth=10, n_estimators=1500;, score=(train=0.767, test=0.689) total time=   6.1s
[CV 1/1] END max_depth=20, n_estimators=500;, score=(train=0.850, test=0.699) total time=   5.1s
[CV 1/1] END max_depth=20, n_estimators=1000;, score=(train=0.850, test=0.698) total time=   6.0s
[CV 1/1] END max_depth=20, n_estimators=1500;, score=(train=0.850, test=0.699) total time=   7.3s
[CV 1/1] END max_depth=30, n_estimators=500;, score=(train=0.919, test=0.701) total time=   5.9s
[CV 1/1] END max_depth=30, n_estimators=1000;, score=(train=0.920, test=0.701) total time=   7.7s
[CV 1/1] END max_depth=30, n_estimators=1500;, score=(train=0.921, test=0.701) total time=   9.2s


In [30]:
clf_nwpe.best_params_

{'max_depth': 30, 'n_estimators': 500}

In [31]:
df_pred_train_nwpe = clf_nwpe.predict(df_features.loc[idx_train, idx_nwpe])
df_pred_test_nwpe = clf_nwpe.predict(df_features.loc[~idx_train, idx_nwpe])

In [32]:
df_score_train = pd.concat([
    df_score_train,
    score_it(df_train['Verdict'], df_pred_train_nwpe, algorithm='RFC', features='N_W_P_E')]
).reset_index(drop=True)

df_score_test = pd.concat([
    df_score_test,
    score_it(df_test['Verdict'], df_pred_test_nwpe, algorithm='RFC', features='N_W_P_E')]
).reset_index(drop=True)

In [33]:
df_score_train

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
1,RFC,N,0.851,0.223,0.405,0.684,0.488,0.527,0.658,0.532,0.621,0.314,0.501,0.562
2,RFC,N_W,0.936,0.490,0.739,0.845,0.808,0.861,0.785,0.808,0.868,0.625,0.761,0.818
3,RFC,N_W_P,0.953,0.709,0.847,0.903,0.911,0.909,0.849,0.896,0.932,0.797,0.848,0.898
4,RFC,N_W_P_E,0.951,0.730,0.896,0.916,0.932,0.900,0.854,0.911,0.942,0.806,0.874,0.912


In [34]:
df_score_test

,algorithm,features,p_NFS,p_UFS,p_CFS,p_wavg,r_NFS,r_UFS,r_CFS,r_wavg,f_NFS,f_UFS,f_CFS,f_wavg
0,RFC,W,0.670,0.600,0.810,0.700,0.990,0.060,0.230,0.680,0.800,0.110,0.350,0.600
1,RFC,N,0.782,0.213,0.479,0.636,0.505,0.554,0.539,0.520,0.614,0.308,0.508,0.550
2,RFC,N_W,0.877,0.439,0.809,0.808,0.816,0.825,0.630,0.767,0.845,0.573,0.708,0.777
3,RFC,N_W_P,0.918,0.643,0.905,0.883,0.920,0.873,0.759,0.872,0.919,0.741,0.826,0.874
4,RFC,N_W_P_E,0.912,0.659,0.935,0.888,0.934,0.865,0.754,0.878,0.923,0.748,0.835,0.879


Adding the NER-labels features we get another improvement in the weighted f1-score.

In [35]:
# Save trained classifier to disk
with bz2.open('../models/rfc_nwpe.bz2', 'wb') as f:
    pickle.dump(clf_nwpe, f)

## Correlations

In [37]:
a = df_features.loc[:, idx_nwpe].copy()
a

,Sentiment,Length,W_000,W_05,W_10,W_100,W_1000,W_10000,W_100000,W_100billion,...,E_money,E_norp,E_ordinal,E_org,E_percent,E_person,E_product,E_quantity,E_time,E_work_of_art
0,0.000000,-0.776708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.456018,-0.147669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.805547,0.874519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
3,0.698942,1.346298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,-0.226299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23525,0.363438,0.481370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.798806,0.0,0.0,0.601589,0.0,0.0,0.0,0.0,0.0,0.0
23526,-0.679982,0.402740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
23527,0.413020,-1.012597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
23528,0.625957,-0.619448,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
corrs = a.corrwith(df['Verdict']).abs()


In [64]:
print(corrs.sort_values(ascending=False)[:10].index.values)

['P_VBD' 'P_VB' 'E_money' 'P_CD' 'P_NNP' 'E_percent' 'P_VBP' 'E_cardinal'
 'Length' 'E_date']


In [57]:
corrs.sort_values(ascending=False)[:10]

P_VBD         0.291801
P_VB          0.270395
E_money       0.226841
P_CD          0.200894
P_NNP         0.193390
E_percent     0.190647
P_VBP         0.185729
E_cardinal    0.181353
Length        0.177743
E_date        0.175472
dtype: float64

In [58]:
corrs.sort_values(ascending=True)[:10]

W_dividend        1.476949e-07
W_old             5.843758e-06
W_sole            7.169514e-06
W_migrat          7.909465e-06
W_hawaii          8.550520e-06
W_minim           1.384480e-05
W_topic           1.421825e-05
W_backfir         1.509875e-05
W_inflationari    1.718115e-05
W_util            1.880420e-05
dtype: float64

### Export results dataframe to Latex for including in report.

In [59]:
to_latex(df_score_test)

\begin{tabular}{llrrrrrrrrrrrr}
\toprule
algorithm & features & p\_NFS & p\_UFS & p\_CFS & p\_wavg & r\_NFS & r\_UFS & r\_CFS & r\_wavg & f\_NFS & f\_UFS & f\_CFS & f\_wavg \\
\midrule
RFC & W & 0.670 & 0.600 & 0.810 & 0.700 & 0.990 & 0.060 & 0.230 & 0.680 & 0.800 & 0.110 & 0.350 & 0.600 \\
RFC & N & 0.782 & 0.213 & 0.479 & 0.636 & 0.505 & 0.554 & 0.539 & 0.520 & 0.614 & 0.308 & 0.508 & 0.550 \\
RFC & N\_W & 0.877 & 0.439 & 0.809 & 0.808 & 0.816 & 0.825 & 0.630 & 0.767 & 0.845 & 0.573 & 0.708 & 0.777 \\
RFC & N\_W\_P & 0.918 & 0.643 & 0.905 & 0.883 & 0.920 & 0.873 & 0.759 & 0.872 & 0.919 & 0.741 & 0.826 & 0.874 \\
RFC & N\_W\_P\_E & 0.912 & 0.659 & 0.935 & 0.888 & 0.934 & 0.865 & 0.754 & 0.878 & 0.923 & 0.748 & 0.835 & 0.879 \\
\bottomrule
\end{tabular}

